# DLW Practical 2
# Multi-label task with MNIST

**Introduction**

In this practical, you can play with multi-label classification using MNIST. Multi-label classification is a generalisation of multi-class classification. Multiple target labels can be assigned to each observation instead of only one like in multi-class classification. This relates to finding a model that maps inputs x to binary vectors y, where each y can take on values between 0 - 1.

**Learning objectives**:

* We have create more labels for the MNIST dataset (i.e., a prime number label and an even number label). You can create more if you like.
* How would you change the output units of the network to accommodate this task? 
* How should the cost function be changed?

In [1]:
%matplotlib inline

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def even_labels(image_labels, one_hot=False):
    even_labels = 1 - image_labels.argmax(axis=1) % 2
    if one_hot:
        return pd.get_dummies(even_labels).as_matrix()
    return even_labels.reshape(-1, 1)

In [3]:
def prime_labels(image_labels, one_hot=False):
    prime_labels = np.in1d(image_labels.argmax(axis=1), [2, 3, 5, 7]).astype(int)
    if one_hot:
        return pd.get_dummies(prime_labels).as_matrix()
    return prime_labels.reshape(-1, 1)

In [4]:
def linear_model(x_dim, n_classes):
    x = tf.placeholder(tf.float32, [None, x_dim])
    W = tf.Variable(tf.random_normal([x_dim, n_classes], stddev=np.sqrt(1. / x_dim)))
    b = tf.Variable(tf.zeros([n_classes]))
    y_ = tf.add(tf.matmul(x, W), b)
    y = tf.placeholder(tf.float32, [None, n_classes])
    return x, W, b, y, y_

In [5]:
def next_batch(num, data, labels):
    idx = np.arange(0, len(data))
    np.random.shuffle(idx)
    idx = idx[:num]
    data_shuffle = [data[i] for i in idx]
    labels_shuffle = [labels[i] for i in idx]
    return np.asarray(data_shuffle), np.asarray(labels_shuffle)

In [8]:
# download MNIST dataset #
mnist = input_data.read_data_sets('../../data/mnist/', one_hot=True)

print('\nTraining: {}\nValidation: {}\nTesting: {}\n'.format(mnist.train.images.shape, mnist.validation.images.shape, mnist.test.images.shape))

train_x, train_y = mnist.train.images, np.concatenate((mnist.train.labels, even_labels(mnist.train.labels, one_hot=False), prime_labels(mnist.train.labels, one_hot=False)), axis=1)
val_x, val_y = mnist.validation.images, np.concatenate((mnist.validation.labels, even_labels(mnist.validation.labels, one_hot=False), prime_labels(mnist.validation.labels, one_hot=False)), axis=1)
test_x, test_y = mnist.test.images, np.concatenate((mnist.test.labels, even_labels(mnist.test.labels, one_hot=False), prime_labels(mnist.test.labels, one_hot=False)), axis=1)

train_examples, x_dim, n_classes = train_x.shape[0], train_x.shape[1], train_y.shape[1]

#########################
# define training cycle #
#########################

epochs = 10
minibatch_size = 50
num_minibatches = int(mnist.train.num_examples / minibatch_size)

print('Epochs: {}\nMini-batch size: {}\nNumber of mini-batches: {}\n'.format(epochs, minibatch_size, num_minibatches))

graph = tf.Graph()

with graph.as_default():

    x, W, b, y, y_ = linear_model(x_dim, n_classes)

    #############################
    # define loss/cost function #
    #############################

    cross_entropy_loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=y_, labels=y))

    # define backpropgagation algorithm/optimiser #
    learning_rate = 0.001
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy_loss)

    ###########################
    # define model evaluation #
    ###########################

    act_class, pred_class = y, tf.round(tf.nn.sigmoid(y_))
    correct_prediction = tf.cast(tf.equal(pred_class, act_class), tf.float32)
    classification_accuracy = tf.reduce_mean(correct_prediction)

    # initializing the variables before starting the session #
    init = tf.global_variables_initializer()
    # launch the graph in a session (use the session as a context manager) #
    with tf.Session() as sess:
        sess.run(init)
        # loop every epoch #
        for epoch in range(epochs):
            # average cost per epoch #
            train_avg_cost = 0.
            train_avg_acc = 0.
            # loop every mini-batch #
            for i in range(num_minibatches):
                # get next mini-batch #
                batch_xs, batch_ys = next_batch(minibatch_size, train_x, train_y)
                # train network #
                sess.run([train_step], feed_dict={x: batch_xs, y: batch_ys})
                # evaluate network #
                train_cross_ent_loss, train_accuracy = sess.run([cross_entropy_loss, classification_accuracy], feed_dict={x: batch_xs, y: batch_ys})
                train_avg_cost += train_cross_ent_loss / num_minibatches
                train_avg_acc += train_accuracy / num_minibatches

            val_cross_ent_loss, val_accuracy = sess.run([cross_entropy_loss, classification_accuracy], feed_dict={x: val_x, y: val_y})

            if epoch % 5 == 0:
                print("Epoch: {}".format(epoch))
                print("Training: {:.3f} (loss), {:.3f} (accuracy)".format(train_avg_cost, train_avg_acc * 100))
                print("Validation: {:.3f} (loss), {:.3f} (accuracy)".format(val_cross_ent_loss, val_accuracy * 100))
                print('--------------------------------------------\n')

        print('Accuracy on test set: {0:.2f}'.format(100 * sess.run(classification_accuracy, feed_dict={x: test_x, y: test_y})))


Extracting ../../data/mnist/train-images-idx3-ubyte.gz
Extracting ../../data/mnist/train-labels-idx1-ubyte.gz
Extracting ../../data/mnist/t10k-images-idx3-ubyte.gz
Extracting ../../data/mnist/t10k-labels-idx1-ubyte.gz

Training: (55000, 784)
Validation: (5000, 784)
Testing: (10000, 784)

Epochs: 10
Mini-batch size: 50
Number of mini-batches: 1100

Epoch: 0
Training: 0.553 (loss), 78.593 (accuracy)
Validation: 0.474 (loss), 84.397 (accuracy)
--------------------------------------------

Epoch: 5
Training: 0.352 (loss), 87.070 (accuracy)
Validation: 0.347 (loss), 87.173 (accuracy)
--------------------------------------------

Accuracy on test set: 87.97
